In [1]:
# To run in jupyter notebook
import nest_asyncio
nest_asyncio.apply()

In [ ]:
import logging
from aiogram import Bot, Dispatcher, executor, types
import inference as inf
from users import *
from censore import *

# init model
tokenizer, model = inf.get_model('../models/GPT2_checkpoint_all_data.pt')
lstm_tokenizer, lstm_model = inf.get_lstm_model('../models/lstm_final_epoch=9-step=439270.ckpt')


with open('tg_api_token.txt') as tg_api:
    TELEGRAM_API_TOKEN = tg_api.read()
    
# Configure logging
logging.basicConfig(level=logging.INFO)


# Initialize bot and dispatcher
bot_state = BotState()
bot = Bot(token=TELEGRAM_API_TOKEN)
dp = Dispatcher(bot)

@dp.message_handler(commands=['help'])
async def send_help(message: types.Message):
    help_message = ''
    with open('help.txt', 'r') as fin:
        help_message = fin.read()
        
    await message.answer(help_message)

@dp.message_handler(commands=['params'])
async def get_params(message: types.Message):
    user = bot_state.get_user_state(message.from_id)
    
    result = f'Ваши параметры генерации: \n{user.params}\n'
    result += f'меньше temperature - менее случайные результаты gpt.\n'
    result += f'больше k - более случайные результаты lstm.'
    
    
    await message.answer(result)
    
@dp.message_handler(commands=['start'])
async def send_start(message: types.Message):
#     user = bot_state.get_user_state(message.from_id)
    await message.reply("Привет! О чём рассказать анекдот?")

@dp.message_handler(commands=['lstm'])
async def send_lstm_result(message: types.Message):
    user = bot_state.get_user_state(message.from_id)
    
    if is_obscene(message.text) is not None:
    
        result = None
        # Фильтр мата
        while result is None:
            result = inf.get_lstm_prediction(f'{message.text}', lstm_tokenizer, lstm_model, \
                                             params_dict=user.params)
            result = is_obscene(result)
            
    else:
        result = 'В запросе есть плохое слово!'
            
    await message.reply(result)
    
@dp.message_handler(commands=['change_param'])
async def change_param(message: types.Message):
    user = bot_state.get_user_state(message.from_id)
    print(user)

    try:
        query = message.text.split(' ')
        param = query[1]
        value = float(query[2])
        
        if param not in user.params.keys():
            result = 'Такого параметра не существует.'
        else:
            user.params[param] = value
        
        result = f'Значение параметра {param} изменено на {value}.'
    except:
        result = 'Запрос должен иметь вид: /change_param <param> <value>'
    
    await message.reply(result)
    
@dp.message_handler()
async def send_gpt_result(message: types.Message):
    user = bot_state.get_user_state(message.from_id)
    
    if is_obscene(message.text) is not None:
    
        result = None
        # Фильтр мата
        while result is None:
            result = inf.get_prediction(f'Расскажи анекдот {message.text}', tokenizer, model, \
                                        params_dict=user.params)
            result = is_obscene(result)
            
    else:
        result = 'В запросе есть плохое слово!'
            
    await message.reply(result)
    await message.answer("О чём ещё рассказать анекдот?")


if __name__ == '__main__':
    executor.start_polling(dp, skip_updates=True)

INFO:aiogram:Bot: russian_jokes_dl [@russian_jokes_dl_bot]
INFO:aiogram.dispatcher.dispatcher:Start polling.


In [1]:
from censore import *

In [2]:
is_obscene('Привет говно')

привет
ривет 
ивет г
вет го
ет гов
т говн
 говно
говно
овно
вно
но
о
привет 
ривет г
ивет го
вет гов
ет говн
т говно
 говно
говно
овно
вно
но
о
привет
ривет 
ивет г
вет го
ет гов
т говн
 говно
говно
овно
вно
но
о
приве
ривет
ивет 
вет г
ет го
т гов
 говн
говно
овно
вно
но
о
привет 
ривет г
ивет го
вет гов
ет говн
т говно
 говно
говно
овно
вно
но
о
приве
ривет
ивет 
вет г
ет го
т гов
 говн
говно
овно
вно
но
о
привет 
ривет г
ивет го
вет гов
ет говн
т говно
 говно
говно
овно
вно
но
о
привет
ривет 
ивет г
вет го
ет гов
т говн
 говно
говно
овно
вно
но
о
приве
ривет
ивет 
вет г
ет го
т гов
 говн
говно
овно
вно
но
о
привет 
ривет г
ивет го
вет гов
ет говн
т говно
 говно
говно
овно
вно
но
о
привет го
ривет гов
ивет говн
вет говно
ет говно
т говно
 говно
говно
овно
вно
но
о
прив
риве
ивет
вет 
ет г
т го
 гов
говн
овно
вно
но
о
приве
ривет
ивет 
вет г
ет го
т гов
 говн
говно
овно
вно
но
о
привет 
ривет г
ивет го
вет гов
ет говн
т говно
 говно
говно
овно
вно
но
о
привет
ривет 
ивет г
вет го
ет г

In [3]:
a.split('|1|-|')[-1]

'"Штирлиц часто подумывал, что это - новая пытка Мюллера.- Не думал, что ты ее придумал, Штирлиц."|'

In [1]:
import inference as inf

In [25]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

def get_model(checkpoint):
    model_name = "DeepPavlov/rudialogpt3_medium_based_on_gpt2_v2"   
    tokenizer =  AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)
    
    model.load_state_dict(torch.load(checkpoint)['model_state_dict'])
    model.eval()
    model.cuda()
    
    return tokenizer, model

def get_prediction(query, tokenizer, model):
    # INFERENCE

    chat_history_ids = torch.zeros((1, 0), dtype=torch.int)
    new_user_input_ids = tokenizer.encode(f"|0|1|{query}{tokenizer.eos_token}", return_tensors="pt")
    chat_history_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1)

    new_user_input_ids = tokenizer.encode(f"|1|-|", return_tensors="pt")
    chat_history_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1)
    input_len = chat_history_ids.shape[-1]

    chat_history_ids = model.generate(
        chat_history_ids.cuda(),
        num_return_sequences=1,                     # use for more variants, but have to print [i]
        max_length=512,
        no_repeat_ngram_size=3,
        do_sample=True,
        top_k=50,
        top_p=0.9,
        temperature = 0.5,                          # 0 for greedy
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
    )
        
    result = f"{tokenizer.decode(chat_history_ids[0].cpu().detach(), skip_special_tokens=True)}"
    
    return result
    

In [2]:
tokenizer, model = get_model('GPT2_checkpoint_all_data.pt')

In [24]:
get_prediction('Расскажи анекдот про Вовочку', tokenizer, model)

'|0|1|Расскажи анекдот про Вовочку|1|3|"- Вовочка, как тебя зовут?- Не знаю, я еще маленький.- Почему?- А зачем?"|'

In [6]:
import numpy as np
import torch

# !git clone https://github.com/dariush-bahrami/character-tokenizer.git
import string
import sys
from charactertokenizer import CharacterTokenizer

# from torch.utils.data import TensorDataset, random_split
# from tqdm import tqdm_notebook

In [14]:
with open('tokenizer_chars.txt', 'r') as fin:
    chars = fin.read()
    
model_max_length = 512
tokenizer = CharacterTokenizer(chars, model_max_length, padding_size='right')

In [15]:
import os
from torch import optim, nn, utils, Tensor
from torchvision.transforms import ToTensor
import lightning.pytorch as pl
import torch.nn as nn
# from transformers import BertModel


# Класс модели (через лайтнинг!)
# LSTM 3 слоя размера 1024 -> два линейных слоя 

class LSTM(pl.LightningModule):
    def __init__(self, n_vocab):
        super().__init__()        
        self.lstm = nn.LSTM(input_size=1, hidden_size=1024, num_layers=3, batch_first=True, dropout=0.2)
        self.dropout = nn.Dropout(0.2)
        self.linear_1 = nn.Linear(1024, 512)
        self.relu = nn.ELU()
        self.linear_2 = nn.Linear(512, n_vocab)
        
        self.loss_fn = nn.CrossEntropyLoss()

    def training_step(self, batch, batch_idx):
        x, y = batch
        x = self.forward(x)
        
        loss = self.loss_fn(x, y)
        self.log("train_loss", loss, prog_bar=True, on_step=False, on_epoch=True)
        return loss
        
    def validation_step(self, batch, batch_idx):
        x, y = batch
        x = self.forward(x)
        
        loss = self.loss_fn(x, y)
        self.log("val_loss", loss, prog_bar=True, on_step=False, on_epoch=True)
        
    def forward(self, x):
        x = x.float().unsqueeze(-1)
        x, _ = self.lstm(x)
        x = x[:, -1, :]
        x = self.linear_1(self.dropout(x))
        x = self.linear_2(self.relu(self.dropout(x)))
        return x

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=2e-4)
        return optimizer

lstm = LSTM(tokenizer.vocab_size)

In [16]:
tokenizer.vocab_size

315

In [17]:
lstm = lstm.load_from_checkpoint(checkpoint_path="../models/lstm_final_epoch=9-step=439270.ckpt", n_vocab=tokenizer.vocab_size)
lstm.cuda()

LSTM(
  (lstm): LSTM(1, 1024, num_layers=3, batch_first=True, dropout=0.2)
  (dropout): Dropout(p=0.2, inplace=False)
  (linear_1): Linear(in_features=1024, out_features=512, bias=True)
  (relu): ELU(alpha=1.0)
  (linear_2): Linear(in_features=512, out_features=315, bias=True)
  (loss_fn): CrossEntropyLoss()
)

In [24]:
k = 2

softmax = nn.Softmax(dim=1)
    
pattern = torch.tensor(tokenizer.encode('Штирлиц'))[:-1]
    
lstm.eval()
lstm.cuda()
print('Prompt: "%s"' % tokenizer.decode(pattern))
with torch.no_grad():
    while True:
        x = pattern / float(tokenizer.vocab_size)
        prediction = lstm(x.unsqueeze(0).cuda().float())
        p = softmax(prediction * k).cpu().flatten().detach().numpy()
        index = np.random.choice(np.arange(len(prediction.flatten())), 1, p=p)
        result = tokenizer.decode(index)
        print(result, end="")
        if index == 1:
            break
        pattern = torch.concatenate((pattern.cuda(), torch.tensor(index).cuda()))
        pattern = pattern[1:]
print()

Prompt: "[CLS]Штирлиц"
 может быть, потом перестальное которого трудно слава 'си' спросил на ней без коробе поворачивается с ней на работать?[SEP]


In [2]:
from lstm_class import *